# Fully-Connected Neural Nets
Implementation of fully connected neural network with a modular approach. Each layer will act as a modular piece implementing it's own `foward` and `backward` methods.

In [5]:
# As usual, a bit of setup

import time
import numpy as np
import matplotlib.pyplot as plt
from classifiers.fc_net import *
from util.data_utils import get_CIFAR10_data
from util.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from util.solver import Solver

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))